<a href="https://colab.research.google.com/github/Danjari/Weather_based_training/blob/main/weatherAppColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

# OpenWeatherMap API configuration
WEATHER_API_KEY = 'YOU_WEATHER_API_KEY'
WEATHER_URL = 'http://api.openweathermap.org/data/2.5/weather'

def get_weather(location):
    """
    Fetches current weather data for a given location (city name).

    Parameters:
        location (str): The city name, optionally with country code (e.g., 'London,UK').

    Returns:
        dict or None: Weather data if successful, else None.
    """
    params = {
        'q': location,
        'appid': WEATHER_API_KEY,
        'units': 'metric'  # Use 'imperial' for Fahrenheit
    }
    try:
        response = requests.get(WEATHER_URL, params=params)
        response.raise_for_status()  # Raises error for bad status
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching weather data: {e}")
        return None

location = input("What's your location? ")
weather_data = get_weather(location)
if weather_data:
    description = weather_data['weather'][0]['description'].title()
    weather_conditions = weather_data['weather'][0]['main'].lower()

    temp = weather_data['main']['temp']
    print(f"\nCurrent weather in {location}: {description},{weather_conditions}, {temp}°C")
else:
    print("\nUnable to fetch weather data. Defaulting to indoor workouts.")

What's your location? New York

Current weather in New York: Clear Sky,clear, 22.65°C


In [ ]:


def determine_workout_type(weather_data):
    """
    Determines whether to suggest indoor or outdoor workouts based on weather data.

    Parameters:
        weather_data (dict): The weather data fetched from OpenWeatherMap.

    Returns:
        str: 'indoor' or 'outdoor'
    """
    if not weather_data:
        return 'indoor'  # Default to indoor if no data

    temp = weather_data['main']['temp']
    weather_conditions = weather_data['weather'][0]['main'].lower()

    #  criteria for outdoor workouts
    if 18 <= temp <= 25 and weather_conditions not in ['rain', 'snow', 'storm', 'drizzle']:
        return 'outdoor'
    else:
        return 'indoor'




def get_exercises(workout_type):
    """
    Fetches exercises based on the workout type.

    Parameters:
        workout_type (str): 'indoor' or 'outdoor'

    Returns:
        list: List of exercise dictionaries.
    """
    EXERCISE_URL = 'https://exercisedb.p.rapidapi.com/exercises'
    EXERCISE_HEADERS = {
        'x-rapidapi-host': 'exercisedb.p.rapidapi.com',
        'x-rapidapi-key': 'YOUR_RAPID_API_KEY'
    }

    try:
        response = requests.get(EXERCISE_URL, headers=EXERCISE_HEADERS)
        response.raise_for_status()
        exercises = response.json()

        # Filter exercises based on workout type
        if workout_type == 'outdoor':
            # no equipments required
            filtered = [ex for ex in exercises if ex['equipment'].lower() in ['body weight', 'none']]


        else:
            # equipement and no equipment ( based on the content of the API)
            filtered = [ex for ex in exercises if ex['equipment'].lower() ]

        return filtered[:10]  # Return top 10 exercises
    except requests.exceptions.RequestException as e:
        print(f"Error fetching exercises: {e}")
        return []


def main():
    """
    Main function to run the Weather-Based Workout Planner.
    """
    location = input("Enter your location (city name, e.g., 'London' or 'New York,US'): ")
    weather_data = get_weather(location)

    if weather_data:
        description = weather_data['weather'][0]['description'].title()
        temp = weather_data['main']['temp']
        print(f"\nCurrent weather in {location}: {description}, {temp}°C")
    else:
        print("\nUnable to fetch weather data. Defaulting to indoor workouts.")

    workout_type = determine_workout_type(weather_data)
    print(f"Recommended workout type: {workout_type.title()}")

    exercises = get_exercises(workout_type)

    if exercises:
        print("\nHere are some exercises for you:")
        for idx, ex in enumerate(exercises, 1):
            #print(f"{idx}. {ex['name'].title()} - {ex['type'].title()} ({ex['equipment'].title()})")
           # print(f"{idx}.{ex} - ")
            print(f"{idx}.{ex['name'].title()} - Targeting your {ex['target']} - Equipment needed: {ex['equipment']} ")
    else:
        print("No exercises found.")

if __name__ == "__main__":
    main()


Enter your location (city name, e.g., 'London' or 'New York,US'): london

Current weather in london: Overcast Clouds, 16.09°C
Recommended workout type: Outdoor

Here are some exercises for you:
1.3/4 Sit-Up - Targeting your abs - Equipment needed: body weight 
2.45° Side Bend - Targeting your abs - Equipment needed: body weight 
3.Air Bike - Targeting your abs - Equipment needed: body weight 
4.All Fours Squad Stretch - Targeting your quads - Equipment needed: body weight 
5.Alternate Heel Touchers - Targeting your abs - Equipment needed: body weight 
6.Alternate Lateral Pulldown - Targeting your lats - Equipment needed: cable 
7.Ankle Circles - Targeting your calves - Equipment needed: body weight 
8.Archer Pull Up - Targeting your lats - Equipment needed: body weight 
9.Archer Push Up - Targeting your pectorals - Equipment needed: body weight 
10.Arm Slingers Hanging Bent Knee Legs - Targeting your abs - Equipment needed: body weight 


In [ ]:
print(get_exercises("indoor"))

[{'bodyPart': 'waist', 'equipment': 'body weight', 'gifUrl': 'https://v2.exercisedb.io/image/POV7GndwlgJ6ws', 'id': '0001', 'name': '3/4 sit-up', 'target': 'abs', 'secondaryMuscles': ['hip flexors', 'lower back'], 'instructions': ['Lie flat on your back with your knees bent and feet flat on the ground.', 'Place your hands behind your head with your elbows pointing outwards.', 'Engaging your abs, slowly lift your upper body off the ground, curling forward until your torso is at a 45-degree angle.', 'Pause for a moment at the top, then slowly lower your upper body back down to the starting position.', 'Repeat for the desired number of repetitions.']}, {'bodyPart': 'waist', 'equipment': 'body weight', 'gifUrl': 'https://v2.exercisedb.io/image/D-k6p91W0xjYIS', 'id': '0002', 'name': '45° side bend', 'target': 'abs', 'secondaryMuscles': ['obliques'], 'instructions': ['Stand with your feet shoulder-width apart and your arms extended straight down by your sides.', 'Keeping your back straight a

In [69]:
# streamlit version of the app
%%writefile app.py
# app.py

import streamlit as st
import requests
st.set_page_config(page_title="Weather-Based Workout Planner", page_icon="💪", layout="centered")

headers ={
    "authorization": st.secrets["WEATHER_API_KEY_VAL"]
}
# OpenWeatherMap API configuration
WEATHER_API_KEY = st.secrets["WEATHER_API_KEY"]  #find a way to hide? DONE
WEATHER_URL = 'http://api.openweathermap.org/data/2.5/weather'


# ExerciseDB API configuration
EXERCISE_URL = 'https://exercisedb.p.rapidapi.com/exercises'
EXERCISE_HEADERS = {
    'x-rapidapi-host': 'exercisedb.p.rapidapi.com',
    'x-rapidapi-key': st.secrets["RAPID_API_KEY"]  # hide DONE
}



def get_weather(location):
    """
    Fetches current weather data for a given location (city name).

    Parameters:
        location (str): The city name, optionally with country code (e.g., 'London,UK').

    Returns:
        dict or None: Weather data if successful, else None.
    """
    params = {
        'q': location,
        'appid': WEATHER_API_KEY,
        'units': 'metric'  # Use 'imperial' for Fahrenheit
    }
    try:
        response = requests.get(WEATHER_URL, params=params)
        response.raise_for_status()  # Raises error for bad status
        return response.json()
    except requests.exceptions.RequestException as e:
        st.error(f"Error fetching weather data: {e}")
        return None

def determine_workout_type(weather_data):
    """
    Determines whether to suggest indoor or outdoor workouts based on weather data.

    Parameters:
        weather_data (dict): The weather data fetched from OpenWeatherMap.

    Returns:
        str: 'indoor' or 'outdoor'
    """
    if not weather_data:
        return 'indoor'  # Default to indoor if no data

    temp = weather_data['main']['temp']
    weather_conditions = weather_data['weather'][0]['main'].lower()

    # Define your criteria for outdoor workouts
    if 15 <= temp <= 25 and weather_conditions not in ['rain', 'snow', 'storm', 'drizzle']:
        return 'outdoor'
    else:
        return 'indoor'

def get_exercises(workout_type):
    """
    Fetches exercises based on the workout type.

    Parameters:
        workout_type (str): 'indoor' or 'outdoor'

    Returns:
        list: List of exercise dictionaries.
    """
    try:
        response = requests.get(EXERCISE_URL, headers=EXERCISE_HEADERS)
        response.raise_for_status()
        exercises = response.json()

        # Filter exercises based on workout type
        if workout_type == 'outdoor':
            # Example: Choose exercises that don't require equipment
            filtered = [ex for ex in exercises if ex['equipment'].lower() in ['body weight', 'none']]
        else:
            # Example: Choose exercises that might require equipment
            filtered = [ex for ex in exercises if ex['equipment'].lower() not in ['body weight', 'none']]

        return filtered[:10]  # Return top 10 exercises
    except requests.exceptions.RequestException as e:
        st.error(f"Error fetching exercises: {e}")
        return []

def main():


    st.title("💪 Weather-Based Workout Planner")
    st.write("Plan your workouts based on the current weather conditions!")

    # Sidebar
    st.sidebar.header("About")
    st.sidebar.info(
        """
        **Weather-Based Workout Planner** suggests workout routines tailored to the current weather of your location.
        - **Indoor Workouts**: Yoga, Strength Training
        - **Outdoor Workouts**: Running, Cycling,Body Training
        """
    )

    # User Input
    location = st.text_input("Enter your location (city name, e.g., 'London' or 'New York,US'):")

    if st.button("Get Workout Recommendations"):
        if location:
            with st.spinner('Fetching weather data...'):
                weather_data = get_weather(location)

            if weather_data:
                description = weather_data['weather'][0]['description'].title()
                temp = weather_data['main']['temp']
                st.success(f"**Current weather in {location}:** {description}, {temp}°C")
            else:
                st.warning("Unable to fetch weather data. Defaulting to indoor workouts.")

            workout_type = determine_workout_type(weather_data)
            st.info(f"**Recommended workout type:** {workout_type.title()}")

            with st.spinner('Fetching exercises...'):
                exercises = get_exercises(workout_type)

            if exercises:
                st.subheader("Here are some exercises for you:")
                for idx, ex in enumerate(exercises, 1):
                    with st.expander(f"Exercise {idx}: {ex['name'].title()}"):
                        st.markdown(f"**Target Muscle Group:** {ex['target'].title()}")
                        st.markdown(f"**Equipment Needed:** {ex['equipment'].title()}")
                        st.image(ex['gifUrl'], caption=f"Exercise{idx}: {ex['name'].title()}", use_column_width=True)
                        st.markdown("**Instructions:**")
                        for instruction in ex['instructions']:
                            st.markdown(f"- {instruction}")
            else:
                st.warning("No exercises found.")
        else:
            st.error("Please enter a valid city name.")

if __name__ == "__main__":
    main()

Overwriting app.py


In [ ]:
!curl ipecho.net/plain

34.150.154.127

In [66]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.150.154.127:8501

your url is: https://afraid-rabbits-marry.loca.lt
  Stopping...
^C


In [62]:
# prompt: install streamlit

!pip install streamlit --quiet
!pip install pyngrok --quiet
!pip install htbuilder --quiet



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3070, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2863, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 5

In [55]:
!npm install -g localtunnel


added 22 packages, and audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [68]:
pip freeze > requirements.txt
